This notebook will look at splitting polygons into equal parts to localize the outlier identification

In [2]:
import os
import glob
import math
from processing_tools import *
from shapely.geometry import MultiPoint
from shapely.geometry import Polygon
from shapely.geometry import Point
from shapely.geometry import LineString
from shapely.ops import triangulate
from shapely.ops import unary_union
from shapely.ops import nearest_points
from shapely.ops import voronoi_diagram
from centerline.geometry import Centerline
from new_methods_2 import *
import math

In [9]:
FZ = gpd.read_file('AZone_48113.shp')
triangles = gpd.read_file(r'U:\klemay\fromAustin\Projects\Hydra\Triangulation\AZone\48113_test\triangles_1m.shp')

azone = FZ.loc[FZ['ZONE'] == 'A']
crs = utm_code(azone)
print(crs)

azone = azone.dissolve().explode().reset_index(0, drop=True)

32614


In [10]:
azone.crs.to_epsg()

4326

In [24]:
for i, a in azone.iterrows():
    print('Polygon: ', i)
    
    a = g(a.geometry, 4326)
    tri = triangles.sjoin(a)

    tri_pts = tri_zpoints(tri)
    
    # Reproject data
    a_prj = a.to_crs(crs)
    tri_pts_prj = tri_pts.to_crs(crs)

    a_center_pts = centerline_pts(a_prj, crs)
    
    quads = vor_quads(a_center_pts.geometry.to_list(), clip=a_prj, crs=crs)

    tri_buff = g(tri_pts_prj.buffer(1), crs)
    bins = quads.sjoin(tri_buff, how='left')
    bins = bins.groupby(level=0)['index_right'].apply(list)

    for b in bins.index:
        pts = tri_pts_prj.loc[tri_pts_prj.index.isin(bins[b])]
        identify_outliers(pts, b)
        print('QC COMPLETE: ', i)
        print('_______________________________________')
    
        

Polygon:  0
                                             geometry  index_right
0   POLYGON ((684271.729 3602963.060, 684271.698 3...          564
0   POLYGON ((684271.729 3602963.060, 684271.698 3...          111
0   POLYGON ((684271.729 3602963.060, 684271.698 3...          162
0   POLYGON ((684271.729 3602963.060, 684271.698 3...          164
0   POLYGON ((684271.729 3602963.060, 684271.698 3...          626
..                                                ...          ...
8   POLYGON ((684738.861 3603184.445, 684739.548 3...          522
8   POLYGON ((684738.861 3603184.445, 684739.548 3...            4
8   POLYGON ((684738.861 3603184.445, 684739.548 3...         1036
8   POLYGON ((684738.861 3603184.445, 684739.548 3...            7
8   POLYGON ((684738.861 3603184.445, 684739.548 3...           10

[527 rows x 2 columns]
Within Range: 0
QC COMPLETE:  0
_______________________________________
Within Range: 1
QC COMPLETE:  0
_______________________________________
Outliers in Bin:

KeyboardInterrupt: 

In [6]:
def tri_zpoints(df):

    crs = df.crs.to_epsg()

    df0 = df[['pt_0_LAT', 'pt_0_LONG', 'pt_0_Z']]
    df1 = df[['pt_1_LAT', 'pt_1_LONG', 'pt_1_Z']]
    df2 = df[['pt_2_LAT', 'pt_2_LONG', 'pt_2_Z']]

    pt0 = gpd.GeoDataFrame(df0, geometry=gpd.points_from_xy(df0['pt_0_LONG'], df0['pt_0_LAT']), crs=crs)[['pt_0_Z', 'geometry']]
    pt1 = gpd.GeoDataFrame(df1, geometry=gpd.points_from_xy(df1['pt_1_LONG'], df1['pt_1_LAT']), crs=crs)[['pt_1_Z', 'geometry']]
    pt2 = gpd.GeoDataFrame(df2, geometry=gpd.points_from_xy(df2['pt_2_LONG'], df2['pt_2_LAT']), crs=crs)[['pt_2_Z', 'geometry']]

    pt0.rename(columns={'pt_0_Z': 'ELEV'}, inplace=True)
    pt1.rename(columns={'pt_1_Z': 'ELEV'}, inplace=True)
    pt2.rename(columns={'pt_2_Z': 'ELEV'}, inplace=True)

    pts = pd.concat([pt0, pt1, pt2], ignore_index=True)

    pts_no_dups = pts.drop_duplicates('geometry')

    return pts_no_dups

In [85]:
#osmnx.utils_geo._quadrat_cut_geometry(geometry, quadrat_width, min_num=3

df = gpd.read_file('a1_poly.shp')
crs = utm_code(df)
crs

'32614'

In [12]:
def centerline_pts(df, crs):
    c = Centerline(df.geometry[0], interpolation_distance=10)
    c = g(c, crs)
    
    c['pts'] = c.apply(lambda x: [x.geometry.interpolate(i/10, normalized=True) for i in range(1, 10)], axis=1)

    return g(c['pts'][0], crs)

In [13]:
def vor_quads(pt_ls, crs, clip=None):
    vor_raw = voronoi_diagram(MultiPoint(pt_ls))
    vor = g(vor_raw, crs).explode().explode() # geometry colleciton --> MultiPolygon --> Polygons

    if clip is not None:
        vor = vor.overlay(clip)
    
    return vor


In [23]:

def identify_outliers(bin0, bid):
    if bin0.std()[0] > 1:
        # Identify the skewness
        hrange = np.abs(bin0.max()[0] - bin0.mean()[0])
        lrange = np.abs(bin0.mean()[0] - bin0.min()[0])
        
        if hrange > lrange:
            print('Outliers in Bin:', bid)
            print(bin0.loc[bin0['ELEV'] > bin0.describe().T['75%'][0]])
        else:
            print('Outliers in Bin:', bid)
            print(bin0.loc[bin0['ELEV'] < bin0.describe().T['25%'][0]])
            
    else:    
        print('Within Range:', bid)

In [86]:
df = df[['FIPS', 'geometry']]
df = df.to_crs(crs)

In [87]:
df

,FIPS,geometry
0,48113,"POLYGON ((684758.104 3603143.727, 684761.217 3..."


In [88]:
center = Centerline(df.geometry[0], interpolation_distance=10)
center = g(center, crs)

In [89]:
center['pts'] = center.apply(lambda x: [x.geometry.interpolate(i/10, normalized=True) for i in range(1, 10)], axis=1)
center

,geometry,pts
0,"MULTILINESTRING ((684193.609 3603059.184, 6841...","[POINT (684218.08273676 3603052.5964880493), P..."


In [90]:
center['pts'][0]

In [91]:
pt_geom = center['pts'][0]
pts = g(pt_geom, crs)


In [92]:
vor_raw = voronoi_diagram(MultiPoint(pt_geom))

In [93]:
vor = g(vor_raw, crs).explode().explode()
vor.head()

geometry
0 0 0  POLYGON ((683757.594 3602522.637, 683757.594 3...
  1 0  POLYGON ((683840.778 3602522.637, 684279.602 3...
  2 0  POLYGON ((684279.602 3603051.849, 684387.178 3...
  3 0  POLYGON ((684351.078 3602522.637, 684396.494 3...
  4 0  POLYGON ((685139.061 3602552.209, 684494.840 3...

In [94]:
vor = vor.overlay(df)



In [135]:
pts = gpd.read_file('zpoints_1m_projected_a1.shp')
pts_buff = g(pts.buffer(1), crs)

In [191]:
bins = vor.sjoin(pts_buff, how='left')
#bins.to_file('bins.shp')
bins

,FIPS,geometry,index_right
0,48113,"POLYGON ((684241.320 3603005.682, 684237.375 3...",53
0,48113,"POLYGON ((684241.320 3603005.682, 684237.375 3...",54
0,48113,"POLYGON ((684241.320 3603005.682, 684237.375 3...",55
0,48113,"POLYGON ((684241.320 3603005.682, 684237.375 3...",340
0,48113,"POLYGON ((684241.320 3603005.682, 684237.375 3...",341
...,...,...,...
8,48113,"POLYGON ((684497.382 3602976.250, 684499.017 3...",17
8,48113,"POLYGON ((684497.382 3602976.250, 684499.017 3...",472
8,48113,"POLYGON ((684497.382 3602976.250, 684499.017 3...",68
8,48113,"POLYGON ((684497.382 3602976.250, 684499.017 3...",501


In [192]:
# groupby bin id
bins = bins.groupby(level=0)['index_right'].apply(list)
bins

0    [53, 54, 55, 340, 341, 56, 342, 57, 295, 9, 29...
1    [8, 468, 304, 303, 51, 78, 79, 339, 52, 80, 81...
2    [42, 43, 331, 44, 332, 45, 333, 46, 49, 76, 13...
3    [6, 23, 22, 24, 313, 72, 321, 352, 73, 473, 31...
4    [179, 178, 239, 177, 238, 498, 176, 241, 437, ...
5    [7, 35, 36, 324, 325, 37, 38, 326, 39, 327, 40...
6    [184, 185, 412, 183, 411, 182, 248, 247, 257, ...
7    [285, 462, 286, 463, 421, 504, 215, 422, 423, ...
8    [14, 307, 471, 16, 308, 67, 347, 121, 120, 119...
Name: index_right, dtype: object

In [254]:
""" the following will be the script used to pipeline and validate the elevation values within each quadrant """
def identify_outliers(bin0, id):
    if bin0.std()[0] > 1:
        hrange = np.abs(bin0.max()[0] - bin0.mean()[0])
        lrange = np.abs(bin0.mean()[0] - bin0.min()[0])
        
        if hrange > lrange:
            print('Outliers in Quad:', id)
            print(bin0.loc[bin0['ELEV'] > bin0.describe().T['75%'][0]])
        else:
            print('Outliers in Quad:', id)
            print(bin0.loc[bin0['ELEV'] < bin0.describe().T['25%'][0]])
            
    else:    
        print('Within Range:', id)


for b in bins.index:
    pts = pts.loc[pts.index.isin(bins[b])]
    identify_outliers(pts, b)

Within Range: 0
Within Range: 1
Within Range: 2
Within Range: 3
Within Range: 4
Within Range: 5
Within Range: 6
Within Range: 7
Within Range: 8
